In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['Simhei']  #显示中文
plt.rcParams['axes.unicode_minus']=False    #显示负号  
# import tikzplotlib
my_dict = {'s': (1,'$\\mathrm{s}$'), 'ms': (1e3,'$\\mathrm{ms}$'), 'us':(1e6,'$\\mathrm{\\mu s}$') ,'ns':(1e9,'$\\mathrm{ns}$')}
# print(my_dict)
plt.rcParams.update({'axes.labelsize': 16,'xtick.labelsize': 14, 'ytick.labelsize': 14,'legend.fontsize': 12})

def mydraw2(filename,figsize=(8,6),title='示波器波形读取',label1='CH1',label2='CH2',x_label='时间',y_label='电压/$\\mathrm{V}$',xrange=(),yrange=((),()),time_unit='s',xychage=False,save=''):
    #同时绘制示波器CH1和CH2的波形
    
    df = pd.read_csv(filename)
    # 提取X轴和Y轴数据
    x_data = df.iloc[1:, 0].to_numpy(dtype=float)
    ch1_data = df.iloc[1:, 1].to_numpy(dtype=float)
    ch2_data = df.iloc[1:, 2].to_numpy(dtype=float)

    dt=df['Increment'][0]
    delt=df['Start'][0]
    # print('delt',delt)
    x_data=x_data*dt+delt
    x_data*=my_dict[time_unit][0]

    if xychage:
        ch1_data,ch2_data=ch2_data,ch1_data

    # 创建两个子图
    fig, (ax1,ax2) = plt.subplots(2, 1, figsize=figsize,sharex=True)
    ylim1,ylim2=yrange
    # 在第一个子图中绘制第一条曲线
    ax1.plot(x_data, ch1_data, label=label1, color='red')
    # ax1.set_xlabel(x_label)
    ax1.set_ylabel(y_label)
    ax1.grid(True)
    if len(ylim1)!=0:
        ymin,ymax=ylim1
        ax1.set_ylim(ymin, ymax)  # 设置y轴范围为0到5
        # ax1.set_yticks(np.linspace(ymin, ymax, 5))  # 设置Y轴刻度值
    if (label1 != None):
        ax1.legend()

    # 在第二个子图中绘制第二条曲线
    ax2.plot(x_data, ch2_data, label=label2, color='blue')
    ax2.set_xlabel(x_label)
    ax2.set_ylabel(y_label)
    ax2.grid(True)
    if len(ylim2)!=0:
        ymin,ymax=ylim2
        ax2.set_ylim(ymin, ymax)  # 设置y轴范围为0到5
        # ax2.set_yticks(np.linspace(ymin, ymax, 5))  # 设置Y轴刻度值
    if (label2 != None):
        ax2.legend()
    plt.xlabel(f'{x_label}/{my_dict[time_unit][1]}')

    if len(xrange) != 0:
        xmin, xmax = xrange
        plt.xlim(xmin, xmax)
    # 设置标题
    plt.suptitle(title)#,fontsize=10)
    fig.align_ylabels([ax1, ax2])
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.subplots_adjust(left=0.1, right=0.9, bottom=0.1, top=0.9, hspace=0.08)

    # 调整子图之间的垂直间距
    # plt.tight_layout()
    if save!='':
        plt.savefig(save,bbox_inches='tight')


    # tikzplotlib.save("test.tex")
    # 显示图表
    plt.show()

    return x_data,ch1_data,ch2_data,time_unit

def mydraw(filename,choice=1,figsize=(8,6),title='示波器波形读取', label='CH1',x_label='时间',y_label='电压/$\\mathrm{V}$',xrange=(), yrange=(), time_unit='s', save=''):
    #绘制示波器CH1或CH2的波形，用choice控制

    # if time_unit == 'us':
    #     time_unit = 'μs'
    
    df = pd.read_csv(filename)

    # 提取X轴和Y轴数据
    x_data = df.iloc[1:, 0].to_numpy(dtype=float)
    ch1_data = df.iloc[1:, choice].to_numpy(dtype=float)

    dt = df['Increment'][0]
    delt = df['Start'][0]

    x_data = x_data * dt + delt
    x_data *= my_dict[time_unit][0]

    # 创建一个图形
    plt.figure(figsize=figsize)

    # 绘制CH1的数据
    plt.plot(x_data, ch1_data, label=label, color='red')

    plt.xlabel(f'{x_label}/{my_dict[time_unit][1]}')
    plt.ylabel(y_label)
    plt.title(title)
    if len(xrange) != 0:
        xmin, xmax = xrange
        plt.xlim(xmin, xmax)

    if len(yrange) != 0:
        ymin, ymax = yrange
        plt.ylim(ymin, ymax)

    plt.grid(True)
    if (label!=None):
        plt.legend()
    plt.tight_layout(rect=[0, 0, 1, 0.95])


    if save != '':
        plt.savefig(save, bbox_inches='tight')
    plt.show()

    return x_data, ch1_data, time_unit

In [2]:
mydict_t = {'s': 1, 'ms': 1e3, 'μs': 1e6 ,'ns':1e9}
mydict_f = {'Hz': 1, 'kHz': 1e-3, 'MHz': 1e-6 }


def mydrawfft(filename,choice=1, title='前置放大器的输出', x_label='时间', y_label='幅值(V)', yrange=(),frange=(), time_unit='s',fre_unit='Hz', save=''):
    
    if time_unit == 'us':
        time_unit = 'μs'
    
    df = pd.read_csv(filename)

    # 提取X轴和Y轴数据
    x_data = df.iloc[1:, 0].to_numpy(dtype=float)
    ch1_data = df.iloc[1:, choice].to_numpy(dtype=float)

    dt = df['Increment'][0]
    delt = df['Start'][0]

    x_data = x_data * dt + delt
    x_data *= mydict_t[time_unit]

    n=len(x_data)
    # 计算FFT
    fft_result = np.fft.fft(ch1_data)
    fft_freqs = np.arange(n)/(n*dt) # FFT频率轴
    fft_freqs *= mydict_f[fre_unit]
    plt.figure(figsize=(12, 6))

    fig, axs = plt.subplots(2, 1, figsize=(12, 6))

    # 上面的子图：波形图
    axs[0].plot(x_data, ch1_data, color='red')
    axs[0].set_xlabel(f'{x_label}/{time_unit}')
    axs[0].set_ylabel(y_label)
    axs[0].set_title(title)
    if len(yrange) != 0:
        ymin, ymax = yrange
        axs[0].set_ylim(ymin, ymax)
    axs[0].grid(True)
    # axs[0].legend()

    axs[1].plot(fft_freqs[1:], np.abs(fft_result)[1:],color='purple')
    axs[1].set_title('FFT变换后')
    axs[1].set_xlabel(f'频率({fre_unit})')
    axs[1].set_ylabel('幅值(V)')
    if len(frange) != 0:
        fmin, fmax = frange
        axs[1].set_xlim(fmin, fmax)
    axs[1].grid(True)

    # 调整子图之间的垂直间距
    fig.align_ylabels([axs[0], axs[1]])
    plt.tight_layout(rect=[0, 0, 1, 0.95])

    if save != '':
        plt.savefig(save, bbox_inches='tight')

    plt.show()


In [3]:
import os

def get_csv_files_sorted_by_mtime(directory):
    files = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith(".csv") and os.path.isfile(os.path.join(directory, file))]
    sorted_files = sorted(files, key=lambda x: os.path.getmtime(x))
    return sorted_files

# 用法示例：
directory_path = "data"
sorted_csv_files = get_csv_files_sorted_by_mtime(directory_path)

for file in sorted_csv_files:
    print(file)

data\200_us_zc.csv
data\200_us_qbc.csv
data\200us_gbc.csv
data\1000us.csv
data\10us.csv
data\1us.csv
data\yj1.csv
data\yj2.csv
data\yj3.csv
data\yj4.csv
data\yj5.csv
data\yj6.csv
data\zy0.csv
data\zy0_5.csv
data\zy1.csv
data\zy2.csv
data\zy3.csv
data\zy4.csv
data\zy5.csv
data\zy6.csv
